# import Lib

In [1]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from math import log10, sqrt
from waifu2x_ncnn_py import Waifu2x

In [2]:
dataset_path = glob.glob('waifu_gan/Waifu-GAN/*')

In [3]:
waifu2x = Waifu2x()

In [ ]:
dataset_path

In [72]:
img_realEsgan_arr =[]
img_waifu_arr = []
img_arr = []
for idx,img_path in enumerate(dataset_path):
    label = img_path.split(os.path.sep)[-1]
    img_name = img_path.split(os.path.sep)[-1]
    
    #original region
    img = load_img(img_path, target_size=(224, 224),interpolation='nearest',color_mode='rgb')
    img = img_to_array(img)/255.
    img_arr.append(img)

    #waifu region
    img_waifu_path = f'output\\waifu2x\\{label}\\{img_name}'

    with Image.open(img_path) as image:
        image = image.convert('RGB')
        image = waifu2x.process_pil(image)
        os.makedirs(f'output\\waifu2x\\{label}', exist_ok=True)
        image.save(img_waifu_path, quality=95)

    img_waifu = load_img(img_waifu_path, target_size=(224, 224),interpolation='nearest',color_mode='rgb')
    img_waifu = img_to_array(img_waifu)/255.
    img_waifu_arr.append(img_waifu)

    #real-esgan region
    img_realEsgan_path = f'output\\real-esgan\\{label}\\{img_name}'

    command = f"realesrgan-ncnn-vulkan.exe -i {img_path} -o {img_realEsgan_path}"
    !{command}

    img_realEsgan = load_img(img_realEsgan_path, target_size=(224, 224),interpolation='nearest',color_mode='rgb')
    img_realEsgan = img_to_array(img_realEsgan)/255.
    img_realEsgan_arr.append(img_realEsgan)

^C


[0 NVIDIA GeForce GTX 1650]  queueC=2[8]  queueG=0[16]  queueT=1[2]
[0 NVIDIA GeForce GTX 1650]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[0 NVIDIA GeForce GTX 1650]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 NVIDIA GeForce GTX 1650]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1
[1 AMD Radeon(TM) Graphics]  queueC=1[2]  queueG=0[1]  queueT=2[1]
[1 AMD Radeon(TM) Graphics]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[1 AMD Radeon(TM) Graphics]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[1 AMD Radeon(TM) Graphics]  subgroup=64  basic=1  vote=1  ballot=1  shuffle=1
0.00%
[0 NVIDIA GeForce GTX 1650]  queueC=2[8]  queueG=0[16]  queueT=1[2]
[0 NVIDIA GeForce GTX 1650]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[0 NVIDIA GeForce GTX 1650]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 NVIDIA GeForce GTX 1650]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1
[1 AMD Radeon(TM) Graphics]  queueC=1[2]  queueG=0[1]  queueT=2[1]
[1 AMD Radeon(TM) Graphics]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[1 AMD 

In [46]:
def PSNR(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
        # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr

In [50]:
def validate_PSNR(img_arr, img_waifu_arr, img_realEsgan_arr):
    psnr_waifu = []
    psnr_realEsgan = []
    for idx, img in enumerate(img_arr):
        psnr_waifu.append(PSNR(img, img_waifu_arr[idx]))
        psnr_realEsgan.append(PSNR(img, img_realEsgan_arr[idx]))

    return psnr_waifu, psnr_realEsgan

In [51]:
validate_PSNR(img_arr, img_waifu_arr, img_realEsgan_arr)

([84.41477661507346, 86.14450806958267],
 [80.32076193913741, 80.69054025898842])

In [15]:
PSNR(img_original,img_realEsgan)

80.32076193913741